In [ ]:
import os, sys
project_dir = os.getcwd()
if project_dir not in sys.path:
    sys.path.append(project_dir)

import skopt
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.svm import SVC
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize

In [ ]:
from dataset import Dataset

dataset_dir = '/home/abian/Data/Dataset/IUMA/DermaDatabase/'
dataset = Dataset(dataset_dir)

In [ ]:
x, y = dataset.get()

In [ ]:
# Test!!

# url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
# df = pd.read_csv(url, header=None)
# data = df.values

# x, y = data[:, :-1], data[:, -1]
# print(np.unique(y))

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
from feature_selection import FeatureSelection, FeatureEquidistantSelection
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV


# pipe = Pipeline([("transform", FeatureEquidistantSelection()), ('svc', SVC())])
pipe = Pipeline([("transform", FeatureSelection()), ('svc', SVC())])

params = dict()
n_features = x.shape[1]
# params['transform__n_features_to_select'] = (8, 34, 'uniform')
params['transform__selected_features'] = Integer(1, float(2**(116)-1), 'log-uniform')
params['svc__C'] = (1e-6, 100.0, 'log-uniform')
params['svc__gamma'] = (1e-6, 100.0, 'log-uniform')
params['svc__degree'] = (1,5)
params['svc__kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the search
search = BayesSearchCV(estimator=pipe, search_spaces=params, n_jobs=-1, cv=cv)
# perform the search
search.fit(x, y)
# report the best result
print(search.best_score_)
print(search.best_params_)

In [ ]:
type(2**(116)-1)

In [ ]:
np.log10(float(2**(116)-1)) / 10

In [ ]:
values = 9642075140
feature_idx = format(values, "b").zfill(34)
feature_idx = np.array(list(map(int, feature_idx)), dtype=np.bool_)
feature_idx

In [ ]:
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1])
rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=2, random_state=36851234)
for train_index, test_index in rskf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]